# Attacking and Defending Active Directory

<a id="index"></a>
## Index


[Global Settings](#globalsettings)

[Python3-winrm](#pythonwinrm)

[Global Functions](#globalfunctions)

[User Enumeration](#userenum)

<a id="globalsettings"></a>
## Global Settings

[Back to Index](#index)

In [1]:
destIP = "192.168.113.161"
parentDir = "/home/kali/hackthebox/output"
box = "Win10"
boxDir = parentDir + "/" + box
!mkdir $boxDir
winrmusername = "thepcn3rd"
#winrmusername = "domain\username"

# Plaintext Password
winrmpassword = "$#Koshka$#"

# With Encrypted Password - Better than plaintext
#Generate the password from a powershell prompt
#$pw = Read-Host -AsSecureString
#$encrypted = ConvertFrom-SecureString -SecureString $pw
#$encrypted

winrmEncryptedPassword = "encryptedpassword"

# Journal
journalPath = boxDir + "/journal.txt"


# Python3 Library for WinRM
# python3-winrm


mkdir: cannot create directory ‘/home/kali/hackthebox/output/Win10’: File exists


<a id="pythonwinrm"></a>
## Python3-winrm

[Back to Index](#index)

Reference: https://www.programcreek.com/python/example/116331/winrm.Session



In [ ]:
import winrm
# In the security logs it shows powershell.exe launching whoami or cmd.exe etc.  Less stealthy...

# Works if the winrm on the remote computer allows an unencrypted Basic Auth password
##session = winrm.Session('computername', auth=('username', 'badpassword'))

# NTLM Auth
session = winrm.Session('computername', auth=('username', 'badpassword'), transport='ntlm')

# CredSSP
#session = winrm.Session('computername', auth=('username', 'badpassword'), transport='credssp')

# Commands to execute using python
result = session.run_ps("whoami; whoami /groups; whoami /priv")
output = (result.std_out).decode('utf-8')
print(output)

<a id="globalfunctions"></a>
## Global Functions

[Back to Index](#index)

In [2]:
def posh(cmd):
    # Uses a plain-text password
    scriptPath = boxDir + "/ps.ps1"
    f = open(scriptPath, "w")
    fj = open(journalPath, "a")
    fj.write("\n\n####### Command: " + cmd + "\n")
    fj.write("################################################   Output  #####################################\n")
    f.write("#!/usr/bin/pwsh\n")
    f.write("$pw = Convertto-Securestring -AsPlainText -Force -String \"" + winrmpassword + "\"\n")
    f.write("$cred = new-object -typename System.Management.Automation.PSCredential -argumentlist \"" + winrmusername + "\",$pw" + "\n")
    f.write("$s = New-PSSession -Computername " + box + " -Authentication Negotiate -Credential $cred\n")
    # Enter pssession $s is a manual way
    f.write("Invoke-Command -Session $s {" + cmd + "}\n")
    f.close()
    !chmod 700 $scriptPath
    output = !$scriptPath
    for line in output:
        print(line)
        fj.write(str(line) + "\n")
    fj.close()
    #print(output)
    print("Completed Commands!")

def poshEP(cmd):
    # Uses an encrypted password
    scriptPath = boxDir + "/ps.ps1"
    f = open(scriptPath, "w")
    fj = open(journalPath, "a")
    fj.write("\n\n####### Command: " + cmd + "\n")
    fj.write("################################################   Output  #####################################\n")
    f.write("#!/usr/bin/pwsh\n")
    f.write("$pwE = ConvertTo-SecureString -String \"" + winrmEncryptedPassword + "\"\n")
    f.write("$cred = new-object -typename System.Management.Automation.PSCredential -argumentlist \"" + winrmusername + "\",$pwE" + "\n")
    f.write("$s = New-PSSession -Computername " + box + " -Authentication Negotiate -Credential $cred\n")
    # Enter pssession $s is a manual way
    f.write("Invoke-Command -Session $s {" + cmd + "}\n")
    f.close()
    !chmod 700 $scriptPath

    output = !$scriptPath
    for line in output:
        print(line)
        fj.write(str(line) + "\n")
    fj.close()
    #print(output)
    print("Completed Commands!")

<a id="userenum"></a>
## User Enumeration

[Back to Index](#index)

In [10]:
poshCommand = "whoami; whoami /groups; whoami /priv"
posh(poshCommand)
#poshEP(poshCommand)

win10\thepcn3rd

GROUP INFORMATION
-----------------

Group Name                                                    Type             SID          Attributes                                                     
============================================================= ================ ============ ===============================================================
Everyone                                                      Well-known group S-1-1-0      Mandatory group, Enabled by default, Enabled group             
NT AUTHORITY\Local account and member of Administrators group Well-known group S-1-5-114    Mandatory group, Enabled by default, Enabled group             
BUILTIN\Administrators                                        Alias            S-1-5-32-544 Mandatory group, Enabled by default, Enabled group, Group owner
BUILTIN\Users                                                 Alias            S-1-5-32-545 Mandatory group, Enabled by default, Enabled group             
NT AUTHORI

### Gather Local Users

In [11]:
poshCommand = "net user"
posh(poshCommand)
#poshEP(poshCommand)


User accounts for \\

-------------------------------------------------------------------------------
Administrator            DefaultAccount           Guest                    
thepcn3rd                WDAGUtilityAccount       
The command completed with one or more errors.

Completed Commands!


<a id="domainenum"></a>
## Domain Enumeration
The computer must have RSAT loaded or you must load the dll for RSAT

[Back to Index](#index)

In [12]:
poshCommand = "$ADClass = [System.DirectoryServices.ActiveDirectory.Domain]; $ADClass::GetCurrentDomain()"
posh(poshCommand)
#poshEP(poshCommand)

MethodInvocationException: Exception calling "GetCurrentDomain" with "0" argument(s): "Current security context is not associated with an Active Directory domain or forest."
Completed Commands!


### Gather Services from Remote Computer

In [13]:
poshCommand = "Get-CimInstance win32_service"
posh(poshCommand)
#poshEP(poshCommand)


ProcessId Name                                     StartMode State   Status Exi
                                                                            tCo
                                                                            de
--------- ----                                     --------- -----   ------ ---
0         AJRouter                                 Manual    Stopped OK     10…
0         ALG                                      Manual    Stopped OK     10…
0         AppIDSvc                                 Manual    Stopped OK     10…
1136      Appinfo                                  Manual    Running OK     0  
0         AppMgmt                                  Manual    Stopped OK     10…
0         AppReadiness                             Manual    Stopped OK     0  
0         AppVClient                               Disabled  Stopped OK     10…
3164      AppXSvc                                  Manual    Running OK     0  
0         AssignedAccessManagerSvc      

<a id="fileBot"></a>
## Powershell Bot - Built off of a file created (non-persistant)

[Back to Index](#index)

Referecne: https://thepcn3rd.blogspot.com/2017/10/powershell-using-add-member-to-add.html

In [23]:
# "createfile" > file.txt
# $file = Get-ChildItem file.txt
## Display all of the members available to the object
# $file | Get-Member 

poshCommand = "$file = Get-ChildItem c:\\users\\thepcn3rd\\documents\\note.txt; $file | Get-Member"
posh(poshCommand)



   TypeName: System.IO.FileInfo

Name                      MemberType     Definition
----                      ----------     ----------                            
LinkType                  CodeProperty   System.String LinkType{get=GetLinkTyp…
Mode                      CodeProperty   System.String Mode{get=Mode;}         
Target                    CodeProperty   System.Collections.Generic.IEnumerabl…
AppendText                Method         System.IO.StreamWriter AppendText()   
CopyTo                    Method         System.IO.FileInfo CopyTo(string dest…
Create                    Method         System.IO.FileStream Create()         
CreateObjRef              Method         System.Runtime.Remoting.ObjRef Create…
CreateText                Method         System.IO.StreamWriter CreateText()   
Decrypt                   Method         void Decrypt()                        
Delete                    Method         void Delete()                         
Encrypt                   Method 